In [ ]:
# Change this if needed to point to your MarkLogic instance.
base_url = "http://localhost"

# The admin account that can be used to create a new user and role.
adminUser = "admin"
adminPassword = "admin"

# The user to create which will be used for all of the examples.
user = "python-blog-user"
password = "pyth0n"
role_name = "python-blog-role"

import json
import requests
from requests.auth import HTTPDigestAuth

admin_session = requests.Session()
admin_session.auth = HTTPDigestAuth(adminUser, adminPassword)

user_session = requests.Session()
user_session.auth = HTTPDigestAuth(user, password)

In [ ]:
# Create a MarkLogic role that allows a user to run all of the examples below. 
# See https://docs.marklogic.com/guide/security/intro for more information on MarkLogic security.

admin_session.post(
    f"{base_url}:8002/manage/v2/roles",
    headers={"Content-type": "application/json"},
    data=json.dumps({
        "role-name":role_name, 
        "role":["tde-admin", "rest-writer", "qconsole-user"], 
        "privilege":[
            {"privilege-name":"xdmp:document-get", "action":"http://marklogic.com/xdmp/privileges/xdmp-document-get", "kind":"execute"}, 
            {"privilege-name":"unprotected-collections", "action":"http://marklogic.com/xdmp/privileges/unprotected-collections", "kind":"execute"}, 
            {"privilege-name":"any-uri", "action":"http://marklogic.com/xdmp/privileges/any-uri", "kind":"execute"}
        ]
    })
)

In [ ]:
# Create a new MarkLogic user with the role that was just created.

admin_session.post(
    f'{base_url}:8002/manage/v2/users',
    headers={'Content-type': 'application/json'},
    data=json.dumps({"user-name": user, "password": password, "role":[role_name]}),
)

In [ ]:
# Load 500 JSON employee documents into the Documents database.

r = requests.get("https://raw.githubusercontent.com/marklogic/marklogic-spark-connector/master/src/test/resources/500-employees.json")

for employee in json.loads(r.text):
    user_session.put(
        f'{base_url}:8000/v1/documents',
        params={'uri': employee['uri'], 'collection': 'employee'},
        data=json.dumps(employee['value'])
    )

print("Finished loading employees.")

In [ ]:
# Search all employee documents, printing out the response from MarkLogic.

r = user_session.get(
    f'{base_url}:8000/v1/search',
    headers={'Accept': 'application/json'},
    params={'collection': 'employee'}
)
print(json.dumps(json.loads(r.text), indent=2))

In [ ]:
# Search employee documents containing "San Jose" and verify only 1 result is returned.

r = user_session.get(
    f'{base_url}:8000/v1/search',
    headers={'Accept': 'application/json'},
    params={'collection': 'employee', 'q': '"San Jose"'}
)
results = json.loads(r.text)
assert results['total'] == 1
print(json.dumps(results, indent=2))

In [ ]:
# Update an employee to contain the phrase "San Jose".

url = f'{base_url}:8000/v1/documents'
uri = '/data/employees/093caccf-f7ed-4572-a8fa-6390caf4d20e.json'

r = user_session.get(url, params={'uri': uri})
doc = json.loads(r.text)
doc['City'] = 'San Jose'
doc['State'] = 'CA'
user_session.put(url, params={'uri':uri}, data=json.dumps(doc))

In [ ]:
# Search for San Jose again and verify that 2 results are now returned.

r = user_session.get(
    f'{base_url}:8000/v1/search',
    headers={'Accept': 'application/json'},
    params={'collection': 'employee', 'q': '"San Jose"'}
)
results = json.loads(r.text)
assert results['total'] == 2
print(json.dumps(results, indent=2))

In [ ]:
# Load a TDE template into the Schemas database. 
# This creates a view projecting rows from the employee documents.
# See https://docs.marklogic.com/guide/app-dev/TDE for more information.

template = {
    "template": {
        "context": "/",
        "collections": ["employee"],
        "rows": [
            {
                "schemaName": "example",
                "viewName": "employee",
                "columns": [
                    {"name": "GivenName", "scalarType": "string", "val": "GivenName"},
                    {"name": "Surname", "scalarType": "string", "val": "Surname"},
                    {"name": "BaseSalary", "scalarType": "double", "val": "BaseSalary"},
                    {"name": "City", "scalarType": "string", "val": "City"},
                    {"name": "Department", "scalarType": "string", "val": "Department"},
                ],
            }
        ],
    }
}

admin_session.put(
    f'{base_url}:8000/v1/documents',
    data=json.dumps(template),
    headers={'Content-Type': 'application/json'},
    params={
        'uri': '/employee-template.json',
        'collection': 'http://marklogic.com/xdmp/tde',
        'database': 'Schemas'
    }
)

In [ ]:
# Retrieve employees as JSON rows using an Optic query, printing the response from MarkLogic.
# See https://docs.marklogic.com/11.0/guide/optic-guide/en/getting-started-with-optic.html for more information.

optic_query = 'op.fromView("example", "employee").where(op.eq(op.col("City"), "San Jose"))'
r = user_session.post(
    f'{base_url}:8000/v1/rows?column-types=header', 
    headers={
        'Content-type': 'application/vnd.marklogic.querydsl+javascript',
        'Accept': 'application/json'
    },
    data=optic_query
)
results = json.loads(r.text)
print(json.dumps(results, indent=2))

In [ ]:
# Retrieve employees as CSV data and create a pandas DataFrame.

%pip install pandas
import pandas as pd
import io

query = 'op.fromView("example", "employee", "").where(op.eq(op.col("City"), "San Jose"))'
r = user_session.post(
    f'{base_url}:8000/v1/rows', 
    headers={
        'Content-type': 'application/vnd.marklogic.querydsl+javascript',
        'Accept': 'text/csv'
    },
    data=query
)

df = pd.read_csv(io.StringIO(r.text))
print(df.head())

In [ ]:
# Cleanup = delete the user and role and the 500 documents.

admin_session.delete(f'{base_url}:8002/manage/v2/roles/{role_name}')
admin_session.delete(f'{base_url}:8002/manage/v2/users/{user}')
admin_session.delete(
    f'{base_url}:8000/v1/search',
    params={'collection': 'employee'}
)